In [1]:
from sklearn.datasets import fetch_openml

In [2]:
data = fetch_openml(name="mnist_784", version = "active")


In [9]:
X = data["data"]
y = data["target"]

In [15]:
#split instances for training and test/val
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=20000)

In [18]:
#Split instances for test and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size = 10000)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC


In [48]:
#Create our models
Logistic_clf = LogisticRegression()
forest_clf = RandomForestClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
Extra_tree = ExtraTreesClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
svc = SVC()


In [51]:
#Logistic_clf.fit(X_train,y_train)
forest_clf.fit(X_train,y_train)
Extra_tree.fit(X_train,y_train)


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=5, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=100,
                     min_weight_fraction_leaf=0.0, n_estimators=150,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [49]:
svc.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [52]:
forest_predictions = forest_clf.predict(X_val)
Extra_tree_predictions = forest_clf.predict(X_val)
svc_predictions = svc.predict(X_val)

In [53]:
from sklearn.metrics import accuracy_score
print(f"Accuracy for forest: {accuracy_score(y_test,forest_predictions)}")
print(f"Accuracy for Extra-trees: {accuracy_score(y_test,Extra_tree_predictions)}")
print(f"Accuracy for svc: {accuracy_score(y_test,svc_predictions)}")

Accuracy for forest: 0.0983
Accuracy for Extra-trees: 0.0983
Accuracy for svc: 0.098


In [54]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[("forest", forest_clf), ("Extra-trees",Extra_tree),("SVC" ,svc)],
                              voting='hard'
                              )

In [55]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('forest',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=5,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=100,
                                                     min_weight_fraction_leaf=0.0,
     

In [56]:
voting_predictions = voting_clf.predict(X_val)

In [57]:
print(f"Accuracy for hard voting: {accuracy_score(y_test,voting_predictions)}")

Accuracy for hard voting: 0.0985


In [58]:
#Let's check with soft voting
from sklearn.ensemble import VotingClassifier
voting_soft_clf = VotingClassifier(estimators=[("forest", forest_clf), ("Extra-trees",Extra_tree),("SVC" ,svc)],
                              voting='soft'
                              )

In [59]:
voting_soft_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('forest',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=5,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=100,
                                                     min_weight_fraction_leaf=0.0,
     

AttributeError: ignored